In [1]:
import os
import pickle
import datetime
import subprocess
import numpy as np

import rmgpy.chemkin
import rmgpy.data.kinetics
# import cantera as ct


import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Load the base model

basedir = '/work/westgroup/harris.se/autoscience/autoscience/butane/models/rmg_model'
# basedir = '/home/moon/autoscience/autoscience/butane/models/rmg_model'
# basedir = '/work/westgroup/harris.se/autoscience/autoscience/butane/sensitivity_analysis/base_rmg24'
# basedir = '/work/westgroup/harris.se/autoscience/autoscience/butane/models/rmg_small_core'
# new_model_dir = '/work/westgroup/harris.se/autoscience/autoscience/butane/improved_models'

basedir = '/work/westgroup/harris.se/autoscience/fuels/butane/small_lib_20230801'

# basedir = '/work/westgroup/harris.se/autoscience/autoscience/debug/s787_rxns/'
new_model_dir = basedir
base_chemkin = os.path.join(basedir, 'chem_annotated.inp')
dictionary = os.path.join(basedir, 'species_dictionary.txt')
transport = os.path.join(basedir, 'tran.dat')
species_list, reaction_list = rmgpy.chemkin.load_chemkin_file(base_chemkin, dictionary_path=dictionary, transport_path=transport)
print(f'{len(species_list)} species, {len(reaction_list)} reactions')


180 species, 3481 reactions


In [3]:
# Load the new kinetics library
DFT_DIR = "/work/westgroup/harris.se/autoscience/autoscience/butane/dft/"
DFT_DIR = "/home/moon/autoscience/autoscience/butane/dft/"
DFT_DIR = "/work/westgroup/harris.se/autoscience/reaction_calculator/dft/"
# kinetics_lib = os.path.join(DFT_DIR, 'harris_butane', 'kinetics', 'reaction.py')
kinetics_lib = os.path.join(DFT_DIR, 'kinetics', 'kinetics')
ark_kinetics_database = rmgpy.data.kinetics.KineticsDatabase()
ark_kinetics_database.load_libraries(kinetics_lib)
# print(ark_kinetics_database.libraries)
print(f'{len(ark_kinetics_database.libraries[""].entries)} entries loaded')

72 entries loaded


In [5]:
ark_kinetics_database.libraries[""].entries[288]

<Entry index=288 label="butane(1) + OH(15) <=> H2O(8) + PC4H9(182)">

In [11]:
str(reaction_list[84])

'OH(15) + butane(1) <=> H2O(8) + [CH2]CCC(24)'

In [8]:
for i in range(0, len(reaction_list)):
    if ark_kinetics_database.libraries[""].entries[288].item.is_isomorphic(reaction_list[i]):
        print(i)

84


In [12]:
change_list = [84]

In [4]:
# load the uncertainty matrix
covariance_file = '/work/westgroup/harris.se/autoscience/autoscience/uncertainty/butane_covariance.pickle'
# covariance_file = '/home/moon/autoscience/autoscience/uncertainty/butane_covariance.pickle'
with open(covariance_file, 'rb') as handle:
    Sigma_k = pickle.load(handle)

uncertainty_cutoff = 3.0
# uncertainty_cutoff = 0.5
# uncertainty_cutoff = 0.0

change_list = []
for i in range(0, len(reaction_list)):
    if Sigma_k[i, i] > uncertainty_cutoff:
        change_list.append(i)



IndexError: index 1822 is out of bounds for axis 0 with size 1822

In [13]:
# stitch each of the library reactions into the mechanism (depends on library index matching indexing in chemkin file)

total_changed = 0
for key in ark_kinetics_database.libraries[''].entries.keys():
    entry = ark_kinetics_database.libraries[''].entries[key]
    index = entry.index
#     print(index, '\t', entry.label, '\t', reaction_list[index])
    for i in range(0, len(reaction_list)):
        if entry.item.is_isomorphic(reaction_list[i]):
            rmg_rxn_index = i
            break
    else:  
        print(f'could not match reaction index {index}')
        continue
  
    
#     # change every possible reaction
#     print(f'changing reaction {rmg_rxn_index} {reaction_list[rmg_rxn_index]}')
#     reaction_list[rmg_rxn_index].kinetics = entry.data
    
    if rmg_rxn_index in change_list:
        print(f'changing reaction {rmg_rxn_index} {reaction_list[rmg_rxn_index]}')
        reaction_list[rmg_rxn_index].kinetics = entry.data
        total_changed += 1

#     if rmg_rxn_index in change_list or rmg_rxn_index in change_list_fs:
#         print(f'changing reaction {rmg_rxn_index} {reaction_list[rmg_rxn_index]}')
#         reaction_list[rmg_rxn_index].kinetics = entry.data
#         total_changed += 1
print(total_changed, 'changed')

could not match reaction index 180
changing reaction 84 OH(15) + butane(1) <=> H2O(8) + [CH2]CCC(24)
could not match reaction index 325
could not match reaction index 326
could not match reaction index 650
could not match reaction index 655
could not match reaction index 894
could not match reaction index 985
could not match reaction index 1003
could not match reaction index 1026
could not match reaction index 1290
could not match reaction index 1301
could not match reaction index 1509
could not match reaction index 1656
could not match reaction index 1287
could not match reaction index 1288
could not match reaction index 1289
could not match reaction index 1326
could not match reaction index 1522
could not match reaction index 1532
could not match reaction index 1578
could not match reaction index 2422
could not match reaction index 2424
could not match reaction index 2425
1 changed


In [ ]:
# Load the old thermo library
DFT_DIR = "/work/westgroup/harris.se/autoscience/autoscience/butane/dft/"
thermo_lib = os.path.join(DFT_DIR, 'thermo', 'thermo')
ark_thermo_database = rmgpy.data.thermo.ThermoDatabase()
ark_thermo_database.load_libraries(thermo_lib)
print(f'{len(ark_thermo_database.libraries["harris_butane"].entries)} entries')

In [ ]:
# stitch each of the library thermos into the mechanism

plot_index = [96]
thermo_change_list = [87, 90, 109, 99, 98, 80, 100, 97, 81, 91, 82, 93]  # 3.0 cutoff
thermo_change_list_fs = [97, 98, 99, 81, 100, 87, 109, 80, 90, 93, 91, 82]  # 3.0 cutoff, flame speeds these are the same

sp_changed = 0
for key in ark_thermo_database.libraries['harris_butane'].entries.keys():
    entry = ark_thermo_database.libraries['harris_butane'].entries[key]
#     print(entry)

    for i, species in enumerate(species_list):
#         if entry.item.is_isomorphic(species.molecule[0]):
        if entry.item.smiles == species.smiles:
            rmg_species_index = i
            break
    else:
        entry_sp = rmgpy.species.Species(smiles=entry.item.smiles)
        for j, species in enumerate(species_list):
            if entry_sp.is_isomorphic(species.molecule[0]):
                rmg_species_index = i
                'matched'
                break
        else:
#             print(f'could not match species index {entry}')
            continue
            raise ValueError (f'could not match species index {entry}')
        
        
    if rmg_species_index in thermo_change_list or rmg_species_index in thermo_change_list_fs:
        print(f'Changing estimated thermo {rmg_species_index}: {species_list[rmg_species_index]}')
        species_list[rmg_species_index].thermo = entry.data
        
        sp_changed += 1
    
print(sp_changed, 'changed')

In [ ]:
# custom save
chemkin_file = os.path.join('/work/westgroup/harris.se/autoscience/autoscience/butane/models/modifications', f'cutoff3_20230418.inp')
# chemkin_file = os.path.join('/home/moon/autoscience/autoscience/butane/models/modifications', f'cutoff3_20230202.inp')
rmgpy.chemkin.save_chemkin_file(chemkin_file, species_list, reaction_list, verbose=True, check_for_duplicates=True)
subprocess.run(['ck2cti', f'--input={chemkin_file}', f'--transport={transport}', f'--output={chemkin_file[:-4]}.cti'])



In [14]:
# custom save 2
chemkin_file = os.path.join('/work/westgroup/harris.se/autoscience/fuels/butane/small_lib_20230801/', f'change288.inp')
# chemkin_file = os.path.join('/home/moon/autoscience/autoscience/butane/models/modifications', f'cutoff3_20230202.inp')
rmgpy.chemkin.save_chemkin_file(chemkin_file, species_list, reaction_list, verbose=True, check_for_duplicates=True)
subprocess.run(['ck2cti', f'--input={chemkin_file}', f'--transport={transport}', f'--output={chemkin_file[:-4]}.cti'])


Wrote CTI mechanism file to '/work/westgroup/harris.se/autoscience/fuels/butane/small_lib_20230801/change288.cti'.
Mechanism contains 180 species and 3490 reactions.
Validating mechanism...PASSED.


CompletedProcess(args=['ck2cti', '--input=/work/westgroup/harris.se/autoscience/fuels/butane/small_lib_20230801/change288.inp', '--transport=/work/westgroup/harris.se/autoscience/fuels/butane/small_lib_20230801/tran.dat', '--output=/work/westgroup/harris.se/autoscience/fuels/butane/small_lib_20230801/change288.cti'], returncode=0)